In [1]:
# Данные по книгам Лабиринт

import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://www.labirint.ru/rating/votes/?page={}'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
products = soup.select('div.product.need-watch')
book_nom = 0
all_books = []

# нахожу данные со всех доступных страниц
for page in range(1, 2): # указать можно больше, чем доступно к просмотру на сайте
    url = base_url.format(page)
    print(f'Парсинг страницы {page}...')

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # проверка на ошибки
        soup = BeautifulSoup(response.text, 'html.parser')

        # сбор данных с текущей страницы
        for product in soup.select('div.product[data-product-id]'):
          product_id = product.get('data-product-id', 'N/A')
          title = product.get('data-name', 'N/A')
          price = product.get('data-price', '0')
          genre = product.get('data-maingenre-name')
          type_ = product.get('data-first-genre-name', )
          discount_price = product.get('data-discount-price', price)
          author = product.find('div', class_='product-author').get_text(strip=True) if product.find('div', class_='product-author') else 'Автор не указан'
          votes_tag = product.find('span', class_='countvotesbg')
          votes = votes_tag.text.strip() if votes_tag else '0'
          reviews_block = product.find('span', class_='product-hint')
          if product_id != 'чему-то':
            book_nom += 1

          # сбор данных по количеству рецензий
          reviews_block = product.find('span', class_='product-hint')
          reviews_count = '0'
          if reviews_block:
              reviews_link = reviews_block.find('a', {'data-event-label': 'reviewsCount'})
              if reviews_link:
                  reviews_span = reviews_link.find('span')
                  if reviews_span:
                      reviews_count = reviews_span.text.strip()
              all_books.append({
                  'ID': product_id,
                  '№': book_nom,
                  'Название': title,
                  'Жанр': genre,
                  'Тип': type_,
                  'Автор': author,
                  'Цена': price,
                  'Цена со скидкой': discount_price,
                  'Рецензии': reviews_count,
                  'Голоса': votes
              })

    except Exception as e:
        print(f'Ошибка на странице {page}: {str(e)}')
        continue

# сохранение в CSV
if all_books:
    df = pd.DataFrame(all_books)
    df.to_csv('labirint_data.csv', index=False, encoding='utf-8-sig')
    print(f'Сохранено {len(df)} записей')

Парсинг страницы 1...
Сохранено 59 записей
